# Quadratic Programming

In [45]:
import sys

sys.path.append('..')

### 1. Schur-Complement Method

##### Example #1.1

$$
\min 3x_1^2 + 2x_1x_2 + x_1x_3 + 2.5x_2^2 + 2x_2x_3 + 2x_3^2 - 8x_1 - 3x_2 - 3x_3, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 + x_3 = 3 \\
x_2 + x_3 = 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        6 & 2 & 1 \\
        2 & 5 & 2 \\
        1 & 2 & 4
    \end{pmatrix},

    c = \begin{pmatrix} 
        -8 \\
        -3 \\
        -3
    \end{pmatrix},

    A = \begin{pmatrix}        
        1 & 0 & 1 \\
        0 & 1 & 1
    \end{pmatrix},

    b = \begin{pmatrix} 
        3 \\
        0
    \end{pmatrix}
$$

In [46]:
import numpy as np

G = np.array([
    [6., 2., 1.],
    [2., 5., 2.],
    [1., 2., 4.]
])

c = np.array([-8., -3., -3.])

A = np.array([
    [1., 0., 1.],
    [0., 1., 1.]
])

b = np.array([3., 0.])

Gi = np.linalg.inv(G)
AGi = A @ Gi
AGiAt = AGi @ A.T
AGigmh = AGi @ c + b

l = np.linalg.solve(AGiAt, AGigmh)

Atlmg = A.T @ l - c

x = Gi @ Atlmg

print("l:", l)
print("x:", x)

l: [ 3. -2.]
x: [ 2. -1.  1.]


### 2. Null-Space Method

##### Example #2.1

$$
\min 3x_1^2 + 2x_1x_2 + x_1x_3 + 2.5x_2^2 + 2x_2x_3 + 2x_3^2 - 8x_1 - 3x_2 - 3x_3, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 + x_3 = 3 \\
x_2 + x_3 = 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        6 & 2 & 1 \\
        2 & 5 & 2 \\
        1 & 2 & 4
    \end{pmatrix},

    c = \begin{pmatrix} 
        -8 \\
        -3 \\
        -3
    \end{pmatrix},

    A = \begin{pmatrix}        
        1 & 0 & 1 \\
        0 & 1 & 1
    \end{pmatrix},

    b = \begin{pmatrix} 
        3 \\
        0
    \end{pmatrix}
$$

In [47]:
G = np.array([
    [6., 2., 1.],
    [2., 5., 2.],
    [1., 2., 4.]
])

c = np.array([-8., -3., -3.])

A = np.array([
    [1., 0., 1.],
    [0., 1., 1.]
])

b = np.array([3., 0.])

AA = np.vstack([
    A,
    np.array([0., 0., 0.])
])

Q, R = np.linalg.qr(AA.T)
detQ = np.linalg.det(Q)

Y = Q[:, :2]
Z = Q[:, 2:]

AY = A @ Y

xy = np.linalg.solve(AY, b)

GZ = G @ Z
ZtGZ = Z.T @ GZ

GY = G @ Y
ZtGY = Z.T @ GY

xz = np.linalg.solve(ZtGZ, -ZtGY @ xy - Z.T @ c)

x = Y @ xy + Z @ xz

l = np.linalg.solve(AY.T, Y.T @ (c + G @ x))

print('AZ:', A @ Z)
print('det Q:', detQ)
print('x:', x)
print('l:', l)

AZ: [[0.]
 [0.]]
det Q: 0.9999999999999996
x: [ 2. -1.  1.]
l: [ 3. -2.]


### 3. The Projected CG Method

##### Example #3.1

$$
\min 3x_1^2 + 2x_1x_2 + x_1x_3 + 2.5x_2^2 + 2x_2x_3 + 2x_3^2 - 8x_1 - 3x_2 - 3x_3, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 + x_3 = 3 \\
x_2 + x_3 = 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        6 & 2 & 1 \\
        2 & 5 & 2 \\
        1 & 2 & 4
    \end{pmatrix},

    c = \begin{pmatrix} 
        -8 \\
        -3 \\
        -3
    \end{pmatrix},

    A = \begin{pmatrix}        
        1 & 0 & 1 \\
        0 & 1 & 1
    \end{pmatrix},

    b = \begin{pmatrix} 
        3 \\
        0
    \end{pmatrix}
$$

In [48]:
from scipy.sparse import csr_matrix

from modules.optimize.constrained.quadratic_programming.equality_constraints import projected_cg

G = csr_matrix([
    [6., 2., 1.],
    [2., 5., 2.],
    [1., 2., 4.]
])

c = np.array([-8., -3., -3.])

A = csr_matrix([
    [1., 0., 1.],
    [0., 1., 1.]
])

b = np.array([3., 0.])

x, l, iter = projected_cg(A, G, c, b, 10**-9, 100)

print(f'x = {x}')
print(f'l = {l}')
print(f'Number of iterations: {iter}')

x = [ 2. -1.  1.]
l = [ 3. -2.]
Number of iterations: 3


### 4. Active-Set Method for Convex QPs

##### Example #4.1

$$
\min x_1^2 + x_2^2 - 2x_1 - 5x_2 + 7.25, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 - 2x_2 + 2 \geq 0 \\
-x_1 - 2x_2 + 6 \geq 0 \\
-x_1 + 2x_2 + 2 \geq 0 \\
x_1 \geq 0 \\
x_2 \geq 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        2 & 0 \\
        0 & 2
    \end{pmatrix},

    c = \begin{pmatrix} 
        -2 \\
        -5
    \end{pmatrix}
$$

<center>
    <h3>Phase I</h3>
</center>

$$
    W_0 = \{2\}
$$

$$
\min z_1 + z_2 + z_3, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 - 2x_2 + z_1 + 2 \geq 0 \\
-x_1 - 2x_2 + z_2 + 6 \geq 0 \\
-x_1 + 2x_2 - z_3 + 2 = 0 \\
x_1, x_2, z_1, z_2, z_3 \geq 0
\end{array}
\right.
$$

<br/>

<center>
    <h3>Standard form</h3>
</center>

$$
\min z_1 + z_2 + z_3, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 - 2x_2 + z_1 - s_1 = -2 \\
-x_1 - 2x_2 + z_2 - s_2 = -6 \\
-x_1 + 2x_2 - z_3 = -2 \\
x_1, x_2, z_1, z_2, z_3, s_1, s_2 \geq 0
\end{array}
\right.
$$

<br/>

$$
    A = \begin{pmatrix}        
        1  & -2  & 1  & 0 & 0  & -1 &  0 \\
        -1 & -2  & 0  & 1 & 0  &  0 & -1 \\
        -1 &  2  & 0  & 0 & -1 &  0 &  0 \\
    \end{pmatrix},

    b = \begin{pmatrix} 
        -2 \\
        -6 \\
        -2
    \end{pmatrix}
$$


In [49]:
from modules.optimize.constrained.linear_programming.interior_points import interior_points

c = np.array([0., 0., 1., 1., 1., 0., 0.])

A = np.array([
    [ 1., -2., 1., 0.,  0., -1.,  0.],
    [-1., -2., 0., 1.,  0.,  0., -1.],
    [-1.,  2., 0., 0., -1.,  0.,  0.]
])

b = np.array([-2., -6., -2.])

x, _, _, obj, iter = interior_points(c, A, b, 10**-9, 100)

print(f'Solution: {x}')
print(f'Objective function: {obj}')
print(f'Number of iterations: {iter}')

Solution: [3.27657085e+00 6.38285425e-01 2.17705333e-10 2.47809680e-10
 5.44881133e-10 4.00000000e+00 1.44685830e+00]
Objective function: 1.0103961460006472e-09
Number of iterations: 11


<center>
    <h3>Phase II</h3>
</center>

$$
\min x_1^2 + x_2^2 - 2x_1 - 5x_2 + 7.25, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 - 2x_2 \geq -2 \\
-x_1 - 2x_2 \geq -6 \\
-x_1 + 2x_2 \geq -2 \\
x_1 \geq 0 \\
x_2 \geq 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        2 & 0 \\
        0 & 2
    \end{pmatrix},

    c = \begin{pmatrix} 
        -2 \\
        -5
    \end{pmatrix}    
$$

<br/>

$$
    A = \begin{pmatrix}        
        1  & -2 \\
        -1 & -2 \\
        -1 &  2 \\
        1  &  0 \\
        0  &  1
    \end{pmatrix},

    b = \begin{pmatrix} 
        -2 \\
        -6 \\
        -2 \\
         0 \\
         0
    \end{pmatrix}
$$

In [50]:
from modules.optimize.constrained.quadratic_programming.inequality_constraints import active_set_method

G = csr_matrix([
    [2., 0.],
    [0., 2.]
])

c = np.array([-2., -5.])

A = csr_matrix([
    [ 1., -2.],
    [-1., -2.],
    [-1.,  2.],
    [ 1.,  0.],
    [ 0.,  1.]
])

b = np.array([-2., -6., -2., 0., 0.])

x = x[0:2]

x, l, iter = active_set_method(A, G, c, b, x, [2], 10**-9, 100)

print(f'x = {x}')
print(f'l = {l}')
print(f'Number of iterations: {iter}')

print(f'Ax = {A @ x}')
print(f'Objective function: {0.5 * np.dot(x, G @ x) + np.dot(c, x) + 7.25}')

x = [1.4 1.7]
l = [0.8]
Number of iterations: 4
Ax = [-2.  -4.8  2.   1.4  1.7]
Objective function: 0.7999999999999989


### 5. Interior-Point Method

##### Example #5.1

$$
\min x_1^2 + x_2^2 - 2x_1 - 5x_2 + 7.25, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
x_1 - 2x_2 \geq -2 \\
-x_1 - 2x_2 \geq -6 \\
-x_1 + 2x_2 \geq -2 \\
x_1 \geq 0 \\
x_2 \geq 0
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        2 & 0 \\
        0 & 2
    \end{pmatrix},

    c = \begin{pmatrix} 
        -2 \\
        -5
    \end{pmatrix}    
$$

<br/>

$$
    A = \begin{pmatrix}        
        1  & -2 \\
        -1 & -2 \\
        -1 &  2 \\
        1  &  0 \\
        0  &  1
    \end{pmatrix},

    b = \begin{pmatrix} 
        -2 \\
        -6 \\
        -2 \\
         0 \\
         0
    \end{pmatrix}
$$

In [51]:
from modules.optimize.constrained.quadratic_programming.inequality_constraints import interior_points

G = np.array([
    [2., 0.],
    [0., 2.]
])

c = np.array([-2., -5.])

A = np.array([
    [ 1., -2.],
    [-1., -2.],
    [-1.,  2.],
    [ 1.,  0.],
    [ 0.,  1.]
])

b = np.array([-2., -6., -2., 0., 0.])

x, l, y, iter = interior_points(A, G, c, b, 10**-9, 100)

print(f'x = {x}')
print(f'l = {l}')
print(f'y = {y}')
print(f'Number of iterations: {iter}')

print(f'Ax = {A @ x}')
print(f'Objective function: {0.5 * np.dot(x, G @ x) + np.dot(c, x) + 7.25}')

x = [1.4 1.7]
l = [8.00000000e-01 7.14768830e-11 4.57037239e-11 6.86317085e-11
 7.72781553e-11]
y = [6.18217753e-11 1.20000000e+00 4.00000000e+00 1.40000000e+00
 1.70000000e+00]
Number of iterations: 12
Ax = [-2.  -4.8  2.   1.4  1.7]
Objective function: 0.8000000000494563


### 6. Gradient Projection Method

##### Example #6.1

$$
\min \frac{1}{2} x^T G x + x^T c, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
0 \leq x_0 \leq 2 \\
1 \leq x_1 \leq 4 \\
-1 \leq x_2 \leq 1 \\
0 \leq x_3 \leq 3 \\
2 \leq x_4 \leq 5
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 1
    \end{pmatrix},

    c = \begin{pmatrix} 
        -1 \\
        -2 \\
        -3 \\
        -4 \\
        -5
    \end{pmatrix}    
$$

In [52]:
from scipy.sparse import eye

from modules.optimize.constrained.quadratic_programming.inequality_constraints import gradient_projection_method

G = eye(5, format='csr')

c = np.array([-1., -2., -3., -4., -5.])

l = np.array([0., 1., -1., 0., 2.])
u = np.array([2., 4.,  1., 3., 5.])

x = (l + u) / 2.

x, s, iter = gradient_projection_method(G, c, l, u, x, 10**-9, 100)

print(f'x = {x}')
print(f'l = {s}')
print(f'Number of iterations: {iter}')


x = [1. 2. 1. 3. 5.]
l = [0.]
Number of iterations: 3


### 7. SLSQP using SciPy library

##### Example #7.1

$$
\min \frac{1}{2} x^T G x + x^T c, \quad \text{subject to} \quad
\left\{
\begin{array}{l}
0 \leq x_0 \leq 2 \\
1 \leq x_1 \leq 4 \\
-1 \leq x_2 \leq 1 \\
0 \leq x_3 \leq 3 \\
2 \leq x_4 \leq 5
\end{array}
\right.
$$

<br/>

$$
    G = \begin{pmatrix}        
        1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 1
    \end{pmatrix},

    c = \begin{pmatrix} 
        -1 \\
        -2 \\
        -3 \\
        -4 \\
        -5
    \end{pmatrix}    
$$

In [53]:
from scipy.optimize import minimize

def f(x: np.ndarray[np.double]) -> np.double:
    return 0.5 * np.dot(x, G @ x) + np.dot(c, x)

bounds = [
    (0, 2),
    (1, 4),
    (-1, 1),
    (0, 3),
    (2, 5)
] 

x0 = np.array([1, 2, 0, 1, 3])

result = minimize(f, x0, method='SLSQP', bounds=bounds)

print(f'x = {result.x}')
print(f'Number of iterations: {result.nit}')

x = [1. 2. 1. 3. 5.]
Number of iterations: 2
